In [195]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [197]:
import pandas as pd

# Load the Excel file from your local system
file_path = 'C:\\Users\\PK\\Downloads\\Canada_Hosp1_COVID_InpatientData.xlsx'
xls = pd.ExcelFile(file_path)

# Load the relevant sheets into dataframes
data_at_admission = pd.read_excel(xls, sheet_name='Data-at-admission')
hospital_length_of_stay = pd.read_excel(xls, sheet_name='Hospital-length-of-stay')

# Define the columns to extract from the data-at-admission sheet (Independent variables)
independent_variables = [
    'age', 'sex', 'height', 'weight', 'comorbidities', 'systolic_blood_pressure', 'diastolic_blood_pressure',
    'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature', 'motor', 'verbal', 'eye', 'wbc', 'rbc',
    'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast',
    'serum_creatinine', 'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2', 'pao2_fio2', 'ph',
    'high_senstivity_cardiac_troponin', 'esr', 'inr', 'ferritin', 'd_dimer', 'crp', 'hs_crp', 'intubated'
]

# Extract the independent variables from the data-at-admission sheet
data_at_admission_filtered = data_at_admission[['id'] + independent_variables]

# Extract the hospital length of stay (dependent variable) from the hospital-length-of-stay sheet
hospital_length_of_stay_filtered = hospital_length_of_stay[['id', 'hospital_length_of_stay']]

# Merge the two sheets on the common 'id' column and name the resulting dataframe as hcap1
hcap1 = pd.merge(data_at_admission_filtered, hospital_length_of_stay_filtered, on='id')

hcap1.head(10)


,id,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,...,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,intubated,hospital_length_of_stay
0,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",99,62,85,20,...,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,No,21
1,2,61,Female,154.9,73.6,"""[\""Hypertension\"",\""Other\""]""",122,77,93,36,...,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,No,5
2,3,58,Female,NaN,NaN,"""[\""Hypertension\""]""",109,72,73,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,7
3,4,94,Male,182.8,66.2,"""[\""Hypertension\"",\""Other\""]""",165,71,92,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9
4,5,91,Male,NaN,NaN,"""[\""Chronic cardiac disease (not hypertension)...",131,76,60,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9
5,6,44,Male,177.0,112.0,"""[\""Hypertension\""]""",130,70,109,50,...,NaN,NaN,NaN,1.1,263.0,1326,234.0,NaN,Yes,32
6,7,50,Male,188.0,91.1,"""[]""",149,100,130,20,...,NaN,NaN,NaN,NaN,NaN,<270,49.5,NaN,No,4
7,8,72,Male,171.0,110.0,"""[\""Chronic cardiac disease (not hypertension)...",92,62,100,24,...,7.35,NaN,NaN,2.5,NaN,1151,NaN,NaN,No,10
8,9,92,Female,148.0,50.4,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",133,78,87,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,11
9,10,66,Female,145.0,53.2,"""[\""Hypertension\"",\""Other\""]""",92,65,116,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,29


In [198]:
hcap1.shape
# no ofrows = 508 
# no of columns = 55  in the preprocessed dataset 

(494, 45)

In [199]:
hcap1.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                494 non-null    int64  
 1   age                               494 non-null    int64  
 2   sex                               494 non-null    object 
 3   height                            264 non-null    float64
 4   weight                            348 non-null    float64
 5   comorbidities                     494 non-null    object 
 6   systolic_blood_pressure           494 non-null    int64  
 7   diastolic_blood_pressure          494 non-null    int64  
 8   heart_rate                        494 non-null    int64  
 9   respiratory_rate                  494 non-null    int64  
 10  oxygen_saturation                 494 non-null    int64  
 11  temperature                       494 non-null    float64
 12  motor   

In [200]:
esr_column = data_at_admission['esr']
print(esr_column.describe())
missing_values = esr_column.isnull().sum()
missing_values

count      5.000000
mean      61.800000
std       34.215494
min       28.000000
25%       32.000000
50%       55.000000
75%       89.000000
max      105.000000
Name: esr, dtype: float64


503

In [201]:
hcap1.mean(numeric_only=True)

id                                   260.248988
age                                   65.973684
height                               166.921591
weight                                80.232471
systolic_blood_pressure              129.378543
diastolic_blood_pressure              75.410931
heart_rate                            97.439271
respiratory_rate                      24.844130
oxygen_saturation                     93.153846
temperature                           37.762348
motor                                  5.910569
verbal                                 4.768293
eye                                    3.943089
wbc                                    8.055691
rbc                                    4.630142
hemoglobin                           129.876016
hematocrit                             0.395108
mcv                                   85.935569
mch                                   28.256504
mchc                                 328.540650
rdw                                   13

In [202]:
# to find the null values 
hcap1.isnull()

,id,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,...,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,intubated,hospital_length_of_stay
0,False,False,False,True,True,False,False,False,False,False,...,True,True,True,False,True,True,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,True,False,False,True,False,False
2,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
4,False,False,False,True,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,True,False,True,True,False,False
490,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,True,True,True,True,False,False
491,False,False,False,True,False,False,False,False,False,False,...,True,True,True,False,True,True,True,True,False,False
492,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,True,False,True,True,False,False


In [203]:
hcap1.isnull().sum()

id                                    0
age                                   0
sex                                   0
height                              230
weight                              146
comorbidities                         0
systolic_blood_pressure               0
diastolic_blood_pressure              0
heart_rate                            0
respiratory_rate                      0
oxygen_saturation                     0
temperature                           0
motor                                 2
verbal                                2
eye                                   2
wbc                                   2
rbc                                   2
hemoglobin                            2
hematocrit                            2
mcv                                   2
mch                                   2
mchc                                  2
rdw                                   2
platelet_count                        2
aptt_aptr                           216


In [204]:
# how many columns have all blank values 
# since number of rows = 494   ,( hs_crp , high_senstiivty_cardiac_troponin) thses have blank values

In [234]:
###one_hot_encoded_data = pd.get_dummies(hcap1, columns = ['sex', 'comorbidities'])
#one_hot_encoded_data.head(5)

,id,age,height,weight,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,...,"comorbidities_""[\""Obesity\"",\""Diabetes\"",\""Other\""]""","comorbidities_""[\""Obesity\"",\""Other\""]""","comorbidities_""[\""Obesity\""]""","comorbidities_""[\""Other\""]""","comorbidities_""[\""Pregnancy\"",\""Other\""]""","comorbidities_""[\""Pregnancy\""]""","comorbidities_""[\""Rheumatologic disorder\"",\""Hematological\"",\""Other\""]""","comorbidities_""[\""Rheumatologic disorder\"",\""Other\""]""","comorbidities_""[\""Stroke\"",\""Other\""]""","comorbidities_""[]"""
0,1,74,NaN,NaN,99,62,85,20,93,37.5,...,False,False,False,False,False,False,False,False,False,False
1,2,61,154.9,73.6,122,77,93,36,88,38.0,...,False,False,False,False,False,False,False,False,False,False
2,3,58,NaN,NaN,109,72,73,24,91,38.3,...,False,False,False,False,False,False,False,False,False,False
3,4,94,182.8,66.2,165,71,92,16,96,37.5,...,False,False,False,False,False,False,False,False,False,False
4,5,91,NaN,NaN,131,76,60,18,95,38.2,...,False,False,False,False,False,False,False,False,False,False


In [240]:
categorical_columns = ['sex', 'comorbidities', 'intubated']

# Perform One-Hot Encoding for categorical columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_categorical = pd.DataFrame(encoder.fit_transform(hcap1[categorical_columns]))

# Restore column names for encoded variables
encoded_categorical.columns = encoder.get_feature_names_out(categorical_columns)

# Remove original categorical columns and concatenate encoded columns
hcap1 = hcap1.drop(columns=categorical_columns)
hcap1 = pd.concat([hcap1, encoded_categorical], axis=1)


KeyError: "None of [Index(['sex', 'comorbidities', 'intubated'], dtype='object')] are in the [columns]"

In [238]:
# data preprocesing , removing null values  ,  dont remove rows replace them with mean values (standard scakar approach)
#also check if featrue scaling is neeeded or not - inidviduAL CHOICE  depends on u 

In [216]:
# Drop one or more columns using df.drop()
columns_to_drop = ['hs_crp','high_senstivity_cardiac_troponin' ]
hcap2=hcap1.drop(columns=columns_to_drop, axis=1)
hcap2.shape


(494, 43)

In [218]:
#hcap2.fillna(hcap2.mean(numeric_only=True), inplace=True)
# Replace missing values in numerical columns with the mean using fillna()
#numerical_columns = hcap2.select_dtypes(include=['float64', 'int64']).columns
#hcap2[numerical_columns] = hcap2[numerical_columns].fillna(hcap2[numerical_columns].mean())
#hcap2.head(10)

In [220]:
#hcap2.shape

In [222]:
hcap1.shape


(494, 45)

In [224]:
hcap2.shape

(494, 43)

In [226]:
one_hot_encoded_data = pd.get_dummies(hcap1, columns = ['sex', 'comorbidities','intubated '])
print(one_hot_encoded_data)

KeyError: "['intubated '] not in index"

In [ ]:
key features likely to impact Hospital Length of Stay (HLOS) include:

Vital Signs:

Blood Pressure, Heart Rate, Respiratory Rate, Oxygen Saturation, and Temperature reflect immediate health status. Abnormalities in these signs often indicate critical conditions, potentially extending the stay.
Laboratory Values:

WBC, Hemoglobin, Platelet Count, Serum Creatinine, Sodium, Potassium, Lactate signal organ function, infection, or inflammation. Abnormal values may require more intensive care and longer stays.
Patient Characteristics:

Age and Comorbidities (e.g., diabetes, hypertension) complicate recovery, leading to prolonged stays, especially in older or chronically ill patients.
Organ Function and Inflammatory Markers:

ALT, AST, PaO2, CRP, Ferritin, D-Dimer indicate liver function, respiratory issues, and systemic inflammation or infection, which can necessitate longer treatment.
Interventions:

Intubation or other critical care needs significantly increase hospital stays.
In short, vital signs, lab results, comorbidities, and the need for intensive care interventions are strong indicators of how long a patient will remain in the hospital.










In [39]:
# try this - 
# Replace missing values with the mean of their respective columns
hcap1.fillna(hcap1.mean(numeric_only=True), inplace=True)

# Verify that missing values have been replaced
print(hcap1.isnull().sum())
# still lot of preprocessing needs to be dones , drop all null columns 

id                                    0
reason_for_admission                  0
age                                   0
sex                                   0
ethnicity                             0
ethnicity_other                     508
height                                0
weight                                0
comorbidities                         0
comorbidities_other                 104
received_covid_vaccine              507
covid_vaccine                       508
medications                           0
smoking_history                     212
year_they_quit                        0
previous_er_visit_within_14_days      0
admission_disposition                 0
systolic_blood_pressure               0
diastolic_blood_pressure              0
heart_rate                            0
respiratory_rate                      0
oxygen_saturation                     0
temperature                           0
motor                                 0
verbal                                0


In [ ]:
# remove outliers - do capping method 
# there is one line code to remove outliers by capping method ( dont remove the extra values but cap them put them inside) - your choice depends on you 

In [ ]:
# ask teacher - do we need to perform eda , any graph , plots as such 
#yes you have to do this 

In [ ]:
#Likely Important Columns (Significant)   -  model weights do this method 
#Age: Older patients often have longer hospital stays due to potential complications and slower recovery.
#Comorbidities: Chronic conditions like diabetes, hypertension, heart disease, and respiratory issues can significantly impact a patient's recovery time.
##Systolic and Diastolic Blood Pressure: These are indicators of cardiovascular health. Abnormal blood pressure can signal acute issues requiring longer hospital treatment.
#Heart Rate: Elevated or irregular heart rates can indicate stress, infection, or cardiac conditions, potentially leading to extended stays.
#Respiratory Rate: A vital sign that reflects the patient's respiratory health. Abnormal respiratory rates are a key indicator of distress, especially relevant in conditions like COVID-19.
#Oxygen Saturation: Low oxygen saturation can indicate respiratory or cardiac issues, often leading to extended hospitalization.

In [ ]:
#  note-  report no longer than 2 pages , only relevant resukts , nothing extra 
# only those columns given in project should be used  ,else remove anything extra 
for commodiites  u need   one hot encoding 
you can  use whatever algothrim u want 
you have to merge two datasets for mlp and diff datasets for rnn 
                                                        you can use keras and tensor flow for mlp and tensor flow for rnn as we ll 